# 生成所需数据

In [1]:
import os
import pandas as pd
import numpy as np
import shutil
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *

In [2]:
# 指定工作空间
env.workspace = r'F:\cache_data\shp_file\qz\ele_qz'
if not os.path.exists(env.workspace):
    os.makedirs(env.workspace)
env.overwriteOutput = True

In [4]:
# 三调地块路径
sd_data = r'F:\cache_data\shp_file\qz\qz_sd_polygon.shp'
# 母质路径
mz_data = r'F:\cache_data\shp_file\qz\dz\rock_type_qz_result_20250228.shp'
# 坡位矢量路径
slope_data = r'F:\cache_data\shp_file\qz\qz_slopepostion_polygon.shp'

In [5]:
# 提取耕林园草
select_name = 'qz_sd_select.shp'
# arcpy.Select_analysis(sd_data,select_name,f"{'DLLB'} IN ('自然土','水稻土', '非自然土')")

In [6]:
# 融合母质
dissolve_name = 'qz_mz_result_dissove.shp'
# arcpy.Dissolve_management(in_features=mz_data,out_feature_class=dissolve_name,dissolve_field='MZMC',multi_part='SINGLE_PART')

In [7]:
# 标识
intersect_name = 'qz_merge_data.shp'
# 使用标识工具而非相交，保持第一个数据集的完整性
# 第一步：先对第一个和第二个数据集执行标识
temp_identity_name = 'qz_identity_temp.shp'
arcpy.Identity_analysis(in_features=select_name, 
                      identity_features=dissolve_name, 
                      out_feature_class=temp_identity_name,
                      join_attributes='ALL')

# 第二步：对中间结果和第三个数据集执行标识
intersect_name = 'qz_merge_data.shp'  # 保持输出名称一致
arcpy.Identity_analysis(in_features=temp_identity_name, 
                      identity_features=slope_data, 
                      out_feature_class=intersect_name,
                      join_attributes='ALL')

<Result 'F:\\cache_data\\shp_file\\qz\\ele_qz\\qz_merge_data.shp'>

In [8]:
# 转单部件
single_name = 'qz_merge_data_single.shp'
arcpy.MultipartToSinglepart_management(intersect_name,single_name)

<Result 'F:\\cache_data\\shp_file\\qz\\ele_qz\\qz_merge_data_single.shp'>